In [15]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("api key : ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)


In [16]:
# import time

index_name = "langchain-test-index" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=384,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# index = pc.Index(index_name)

In [17]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [20]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
extracted_data = load_pdf("./data/")
text_chunks = text_split(extracted_data)
embeddings = download_hugging_face_embeddings()


C:\Users\moham\AppData\Local\Temp\ipykernel_21972\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import Pinecone
docsearch=Pinecone.from_texts(
    [t.page_content for t in text_chunks],
    embeddings,
    index_name=index_name
)

In [ ]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_existing_index(index_name, embeddings)

# query = "What are Allegries"
# docs = docsearch.similarity_search(query, k = 3)
# print("Result", docs)

In [12]:
from langchain.llms import CTransformers
llm = CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                      model_type="llama",
                      config={'max_new_tokens':512,
                              'temperature':0.8})

In [42]:
# from langchain.chains import RetrievalQA

# qa = RetrievalQA.from_chain_type(
#         llm=llm,
#         chain_type="stuff", 
#         retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
#         return_source_documents=True,
#         chain_type_kwargs=chain_type_kwargs_t
#     )

In [ ]:
# qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt_template)

# qa = RetrievalQA(
#     retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
#     chain_type_kwargs=qa_chain,
#     return_source_documents=True
# )

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

prompt_template = PromptTemplate(
    input_variables=["context", "question"], 
    template="""
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful answer:
"""
)

chain_type_kwargs = {"prompt": prompt_template}

qa_chain = load_qa_chain(llm, chain_type="stuff", **chain_type_kwargs)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs 
)

C:\Users\moham\AppData\Local\Temp\ipykernel_21972\166571942.py:23: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff", **chain_type_kwargs)


In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])